In [1]:
import pandas as pd
import sqlite3

In [26]:
connection = sqlite3.connect("database/enfermedades_db.db")
print (connection.total_changes)
cursor = connection.cursor()


0


In [19]:
SELECT ID_REGISTRO, EDAD_ANOS, DICTAMEN, ESTATUS_CASO, dictamen.DESCRIPCIÓN, estatus_caso.DESCRIPCIÓN
FROM registro
INNER JOIN dictamen ON registro.DICTAMEN = dictamen.clave
INNER JOIN estatus_caso ON registro.ESTATUS_CASO = estatus_caso.clave;

,FECHA_ACTUALIZACION,ID_REGISTRO,SEXO,EDAD_ANOS,ENTIDAD_RES,MUNICIPIO_RES,HABLA_LENGUA_INDIG,INDIGENA,ENTIDAD_UM_NOTIF,MUNICIPIO_UM_NOTIF,...,INMUNOSUPR,CIRROSIS_HEPATICA,EMBARAZO,DEFUNCION,DICTAMEN,TOMA_MUESTRA,RESULTADO_PCR,ESTATUS_CASO,ENTIDAD_ASIG,MUNICIPIO_ASIG
0,2023-10-02,852727,1,50,16,6,2,2,16,6,...,2,2,2,2,5.0,1,5,3,16,6
1,2023-10-02,1001218,2,17,6,2,2,2,6,2,...,2,2,2,2,5.0,1,5,3,6,2
2,2023-10-02,1001227,2,2,31,102,2,2,31,102,...,2,2,2,2,5.0,2,5,1,31,102
3,2023-10-02,1001243,1,69,20,67,2,2,20,67,...,2,2,2,2,5.0,2,5,1,20,67
4,2023-10-02,1001248,1,22,27,4,2,2,27,4,...,2,2,1,2,5.0,1,3,2,27,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49995,2023-10-02,1045553,1,24,17,7,2,2,17,999,...,2,2,2,2,5.0,2,5,1,17,7
49996,2023-10-02,1076990,1,11,21,25,2,2,21,25,...,2,2,2,2,5.0,1,4,2,21,25
49997,2023-10-02,1044826,1,57,21,43,1,1,21,43,...,2,2,2,2,5.0,1,5,3,21,43
49998,2023-10-02,1049900,2,43,21,43,1,1,21,43,...,2,2,2,2,5.0,1,5,3,21,43


In [15]:
df = pd.read_sql_query("SELECT * FROM registro", connection)
df1 = pd.read_sql_query("SELECT * FROM dictamen", connection)
df1['DICTAMEN'] = df1['CLAVE']
df1

,CLAVE,DESCRIPCIÓN,DICTAMEN
0,1,CONFIRMADO A DENGUE,1
1,2,CONFIRMADO A CHIKUNGUNYA,2
2,3,NEGATIVO,3
3,4,EN ESTUDIO,4
4,5,NO APLICA,5


In [28]:
print(cursor.fetchall())

[]


In [31]:
cursor.execute("SELECT * FROM Relacion_Edad;") 
rows = cursor.fetchall()

OperationalError: no such table: Relacion_Edad

In [18]:
second_query = "SELECT ID_REGISTRO, EDAD_ANOS, DICTAMEN, ESTATUS_CASO, dictamen.DESCRIPCIÓN, estatus_caso.DESCRIPCIÓN FROM registro INNER JOIN dictamen ON registro.dictamen = dictamen.clave;"
second_query = second_query
SELECT ID_REGISTRO, EDAD_ANOS, DICTAMEN, ESTATUS_CASO, dictamen.DESCRIPCIÓN, estatus_caso.DESCRIPCIÓN
FROM registro
INNER JOIN dictamen ON registro.DICTAMEN = dictamen.clave
INNER JOIN estatus_caso ON registro.ESTATUS_CASO = estatus_caso.clave;
df2 = pd.read_sql_query(second_query, connection)

In [25]:
connection.close()

In [20]:
df = pd.merge(df, df1, 'inner', on = 'DICTAMEN')

,FECHA_ACTUALIZACION,ID_REGISTRO,SEXO,EDAD_ANOS,ENTIDAD_RES,MUNICIPIO_RES,HABLA_LENGUA_INDIG,INDIGENA,ENTIDAD_UM_NOTIF,MUNICIPIO_UM_NOTIF,...,EMBARAZO,DEFUNCION,DICTAMEN,TOMA_MUESTRA,RESULTADO_PCR,ESTATUS_CASO,ENTIDAD_ASIG,MUNICIPIO_ASIG,CLAVE,DESCRIPCIÓN
0,2023-10-02,852727,1,50,16,6,2,2,16,6,...,2,2,5.0,1,5,3,16,6,5,NO APLICA
1,2023-10-02,1001218,2,17,6,2,2,2,6,2,...,2,2,5.0,1,5,3,6,2,5,NO APLICA
2,2023-10-02,1001227,2,2,31,102,2,2,31,102,...,2,2,5.0,2,5,1,31,102,5,NO APLICA
3,2023-10-02,1001243,1,69,20,67,2,2,20,67,...,2,2,5.0,2,5,1,20,67,5,NO APLICA
4,2023-10-02,1001248,1,22,27,4,2,2,27,4,...,1,2,5.0,1,3,2,27,4,5,NO APLICA
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49993,2023-10-02,1053518,1,65,17,11,2,2,17,7,...,2,1,1.0,2,5,2,17,11,1,CONFIRMADO A DENGUE
49994,2023-10-02,1072407,1,39,17,11,2,2,17,7,...,2,1,1.0,2,5,2,17,11,1,CONFIRMADO A DENGUE
49995,2023-10-02,1075520,1,46,17,11,2,2,17,8,...,2,1,1.0,1,3,2,17,11,1,CONFIRMADO A DENGUE
49996,2023-10-02,1049728,1,30,17,14,2,2,17,7,...,2,1,1.0,1,2,2,17,14,1,CONFIRMADO A DENGUE


In [6]:
connection.close()